In [ ]:
#Project initially compiling information about beer habits from untappd API, with possibility of making recommendation system

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import requests
import http.client 
import json
import pprint
import pandas as pd
# import geopandas as gdp
import numpy as np
from scipy import stats
from scipy.stats import t
from datetime import datetime
import csv
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
!pip3 install geopandas

In [ ]:
!conda install -c conda-forge gdal -y

In [ ]:
import sys
sys.executable

In [ ]:
!pip3 install pyproj

In [ ]:
!pip3 install wheel

!pip3 install numpy
!pip3 install pandas
!pip3 install shapely
!pip3 install gdal
!pip3 install fiona
!pip3 install pyproj
!pip3 install six
!pip3 install rtree
!pip3 install geopandas

In [ ]:
f = open('client.txt')
for line_counter, line in enumerate(f.readlines()):
#     print(line)
    if(line_counter == 0):
        client_id = str(line).strip()
    if(line_counter == 1):
        client_secret = str(line).strip()
print(client_id)       
print(client_secret)
f.close()


In [ ]:
url = f"https://api.untappd.com/v4/user/info/gregavola?client_id={client_id}&client_secret={client_secret}"
print(url)
# https://api.untappd.com/v4/user/checkins/gregavola?client_id=6D2AAB5EC9DDD9F3EE8289B7D899F9AEA36E4BBB&client_secret=AA7B434A5516D89F95F1E950BCFA663245C32BBF

In [ ]:
print("hello")

In [ ]:
response = requests.request("GET", url)#, auth = (client_id, client_secret)) 
response_json = response.json()
print(response.status_code)

In [ ]:
pprint.pprint(response_json["response"]["user"]["checkins"])

In [ ]:
def getUserInfo(username):
    url = f"https://api.untappd.com/v4/user/info/{username}?client_id={client_id}&client_secret={client_secret}"
    
    response = requests.request("GET", url) 
    if(response.status_code == 404):
        print("Connection not made, returning nothing")
        return 0
    if(response.status_code == 200):      
        print(f"Response code is {response.status_code}, all is well")
    response_json = response.json()
    
    return response_json
    

In [ ]:
getUserInfo("gregavola")

In [ ]:
def getDistinctBeers(username):
    url = f"https://api.untappd.com/v4/user/beers/{username}?client_id={client_id}&client_secret={client_secret}"
    
    response = requests.request("GET", url) 
    if(response.status_code == 404):
        print("Connection not made, returning nothing")
        return 0
    if(response.status_code == 200):      
        print(f"Response code is {response.status_code}, all is well")
    response_json = response.json()
    
    return response_json
    

In [ ]:
distinct_beers_json = getDistinctBeers("conorhamill36")

In [ ]:
pprint.pprint(distinct_beers_json["response"])

In [ ]:
# pprint.pprint(distinct_beers_json["response"]["beers"]["items"])
print(len(distinct_beers_json["response"]["beers"]["items"]))

In [ ]:
# for beer in distinct_beers_json["response"]["beers"]["items"]:
# #     pprint.pprint(beer["beer"])
#     pprint.pprint(beer["beer"]["beer_abv"])
#     pprint.pprint(beer["beer"]["beer_ibu"])
#     pprint.pprint(beer["beer"]["beer_style"])
#     pprint.pprint(beer["beer"]["rating_score"])
next_url = distinct_beers_json["response"]["pagination"]["next_url"]    

In [ ]:
#Returns dataframe of information on user's beers
def getUsersDistinctBeers(username, pages = 1):
    users_distinct_beer_data = []
    users_distinct_beer_data_headings = []
    
    url = f"https://api.untappd.com/v4/user/beers/{username}?limit=50&client_id={client_id}&client_secret={client_secret}"

    for page in range(0, pages):
        print(page)
        if(page > 0):
            url = f"https://api.untappd.com/v4/user/beers/{username}?limit=50&offset={int(page*50)}&client_id={client_id}&client_secret={client_secret}"
        print(f"url is {url}") 
        response = requests.request("GET", url) 
        
        if(response.status_code == 404):
            print("Connection not made, returning nothing")
            return 0
        if(response.status_code == 200):      
            print(f"Response code is {response.status_code}, all is well")
        distinct_beers_json = response.json()
        

        #return response_json
        for beer in distinct_beers_json["response"]["beers"]["items"]:
#             pprint.pprint(beer["beer"])
            beer_abv = beer["beer"]["beer_abv"]
            beer_ibu = beer["beer"]["beer_ibu"]
            beer_style = beer["beer"]["beer_style"]
            rating_score = beer["beer"]["rating_score"]
            user_rating_score = beer["rating_score"]
            brewery_id = beer["brewery"]["brewery_id"]
            brewery_name = beer["brewery"]["brewery_name"]
            country_name = beer["brewery"]["country_name"]
            brewery_lat = beer["brewery"]["location"]["lat"]
            brewery_lng = beer["brewery"]["location"]["lng"]
                        
            print(beer_abv, beer_ibu, beer_style, rating_score)
            users_distinct_beer_data.append([beer_abv, beer_ibu, beer_style, rating_score, user_rating_score, brewery_id, brewery_name, country_name, brewery_lat, brewery_lng])
    users_distinct_beer_data_headings = ['beer_abv', 'beer_ibu', 'beer_style', 'rating_score', "user_rating_score", "brewery_id", "brewery_name", "country_name", "brewery_lat", "brewery_lng"]
    
    return users_distinct_beer_data, users_distinct_beer_data_headings

In [ ]:
users_distinct_beer_data, users_distinct_beer_data_headings = getUsersDistinctBeers("conorhamill36", pages = 5)

In [ ]:
# print(users_distinct_beer_data_headings)
beer_df = pd.DataFrame(users_distinct_beer_data, columns = users_distinct_beer_data_headings)
beer_df

In [ ]:
beer_df.plot.scatter(x = 'beer_abv', y = 'beer_ibu', legend = 'beer_style')

In [ ]:
#Need to call matplotlib for each group
plt.scatter(x = beer_df['beer_abv'], y = beer_df['beer_ibu'], legend = beer_df['beer_style'])

In [ ]:
beer_df.plot.scatter(x = 'rating_score', y = 'user_rating_score')

In [ ]:
beer_df.describe()

In [ ]:
beer_df['country_name'].value_counts()

In [ ]:
#Display brewery locations with geopandas
world = gpd.read_file(geopandas.datasets.get_path('naturalearth_lowres'))


In [ ]:
#Group together beer types